In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
pd.set_option('display.max_columns', None)
import numpy as np

data = pd.read_excel('nba_stats.xlsx')

In [3]:
data.shape

(3821, 27)

Data Cleaning

In [4]:
data.drop(columns=['RANK','EFF'], inplace=True)

In [5]:
data['season_start_year'] = data['Year'].str[:4].astype(int) # create a new column for the start year of the season as an integer

In [6]:
data['TEAM'].replace(to_replace=['NOP','NOH'], value='NO', inplace=True) # The hornets and pels are the same franchise

C:\Users\user\AppData\Local\Temp\ipykernel_7812\639342632.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['TEAM'].replace(to_replace=['NOP','NOH'], value='NO', inplace=True) # The hornets and pels are the same franchise


In [7]:
rs_df = data[data['Season_type']=='Regular Season']
playoffs_df = data[data['Season_type']=='Playoffs']

In [8]:
data.columns

Index(['Year', 'Season_type', 'PLAYER_ID', 'PLAYER', 'TEAM_ID', 'TEAM', 'GP',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PTS',
       'season_start_year'],
      dtype='object')

In [9]:
total_cols = ['MIN','FGM','FGA','FG3M','FG3A','FTM','FTA',
              'OREB','DREB','REB','AST','STL','BLK','TOV','PTS']

Which player stats are correlated?

In [10]:
data_per_min = data.groupby(['PLAYER','PLAYER_ID','Year'])[total_cols].sum().reset_index() #sum the totals for each player for each year
for col in data_per_min.columns[4:]: # divide the totals by the minutes played to get per minute stats
    data_per_min[col] = data_per_min[col]/data_per_min['MIN']
#relevant % and advanced stats for the analysis
data_per_min['FG%'] = data_per_min['FGM']/data_per_min['FGA']
data_per_min['3PT%'] = data_per_min['FG3M']/data_per_min['FG3A']
data_per_min['FT%'] = data_per_min['FTM']/data_per_min['FTA']
data_per_min['FG3A%'] = data_per_min['FG3A']/data_per_min['FGA']
data_per_min['PTS/FGA'] = data_per_min['PTS']/data_per_min['FGA']
data_per_min['FG3M/FGM'] = data_per_min['FG3M']/data_per_min['FGM']
data_per_min['FTA/FGA'] = data_per_min['FTA']/data_per_min['FGA']
data_per_min['TRU%'] = 0.5*data_per_min['PTS']/(data_per_min['FGA']+0.475*data_per_min['FTA'])
data_per_min['AST_TOV'] = data_per_min['AST']/data_per_min['TOV']
# filter out players with less than 50 minutes played
data_per_min = data_per_min[data_per_min['MIN']>=50]
data_per_min.drop(columns='PLAYER_ID', inplace=True)

# Ensure data_per_min contains only numeric columns for correlation calculation
numeric_data_per_min = data_per_min.select_dtypes(include=[np.number])

fig = px.imshow(numeric_data_per_min.corr())
fig.show()

A few interesting points:
1. There is a -0.04 ratio between AST/TOV ratio and TRU%. That is surprising because you would expect that the more "responsible" players would be more efficient (taking better shots, not forcing plays), and it doesn't show in the correlation.

2. There is a -0.14 ratio between PTS and FG3A%, I would expect the ratio to be positive - the more 3s you take == the more points you score. It might be related to the fact that there are many role players, who shoot less than the star players and hence score less points, but most of the shots they are taking are 3s

3. there is a relatively high correlation (0.31) between AST/TOV ratio to steals - shows that smart decision making translates the same on offense and defense - being a great athlete is not enough for being a good defensive player

How are mins played distributed?

Shows how rotations are more strict in the playoffs compared to the RS

In [16]:
fig = px.histogram(x=rs_df['MIN']) # plot a histogram of minutes played in the RS
fig.show()

In [14]:
fig = px.histogram(x=playoffs_df['MIN']) # plot the distribution of minutes played in the playoffs
fig.show()

How has the game changed over the past 10 years?

In [23]:
# create a new column for the total number of possessions estimated using the formula
change_df = data.groupby('season_start_year')[total_cols].sum().reset_index()
#total possessions = FGA - OREB + TOV + 0.44*FTA
change_df['POSS_est'] = change_df['FGA']-change_df['OREB']+change_df['TOV']+0.44*change_df['FTA']
# add the per minute stats to the change_df
change_df = change_df[list(change_df.columns[0:2])+['POSS_est']+list(change_df.columns[2:-1])]

change_df['FG%'] = change_df['FGM']/change_df['FGA']
change_df['3PT%'] = change_df['FG3M']/change_df['FG3A']
change_df['FT%'] = change_df['FTM']/change_df['FTA']
change_df['AST%'] = change_df['AST']/change_df['FGM']
change_df['FG3A%'] = change_df['FG3A']/change_df['FGA']
change_df['PTS/FGA'] = change_df['PTS']/change_df['FGA']
change_df['FG3M/FGM'] = change_df['FG3M']/change_df['FGM']
change_df['FTA/FGA'] = change_df['FTA']/change_df['FGA']
change_df['TRU%'] = 0.5*change_df['PTS']/(change_df['FGA']+0.475*change_df['FTA'])
change_df['AST_TOV'] = change_df['AST']/change_df['TOV']

change_df

,season_start_year,MIN,POSS_est,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PTS,FG%,3PT%,FT%,AST%,FG3A%,PTS/FGA,FG3M/FGM,FTA/FGA,TRU%,AST_TOV
0,2012,8404.6,3348.172,1298.8,2874.7,253.5,710.2,626.2,831.3,377.2,1080.8,1456.9,737.0,260.6,174.4,484.9,3476.4,0.451804,0.356942,0.753278,0.567447,0.247052,1.209309,0.195180,0.289178,0.531630,1.519901
1,2013,10557.4,4206.136,1641.1,3613.3,342.7,956.5,815.1,1071.9,467.9,1368.1,1835.6,934.4,321.3,199.8,589.1,4435.7,0.454183,0.358285,0.760425,0.569374,0.264716,1.227604,0.208823,0.296654,0.537993,1.586148
2,2014,10453.0,4201.076,1633.5,3648.9,345.5,987.5,766.6,1022.9,475.5,1426.9,1898.5,946.3,330.7,214.3,577.6,4375.1,0.447669,0.349873,0.749438,0.579308,0.270630,1.199019,0.211509,0.280331,0.529061,1.638331
3,2015,10432.3,4202.760,1624.2,3638.0,372.1,1060.3,763.5,1014.0,455.3,1434.1,1888.0,916.6,331.4,215.9,573.9,4382.5,0.446454,0.350938,0.752959,0.564339,0.291451,1.204645,0.229097,0.278725,0.531902,1.597142
4,2016,10489.6,4313.692,1701.7,3713.4,433.3,1211.5,807.5,1039.3,433.4,1432.2,1862.9,982.4,331.6,203.1,576.4,4642.8,0.458259,0.357656,0.776965,0.577305,0.326251,1.250283,0.254628,0.279878,0.551786,1.704372
5,2017,6293.6,2602.492,1046.6,2250.9,275.0,751.9,440.8,571.8,253.4,880.3,1133.1,605.1,203.0,126.4,353.4,2809.8,0.464970,0.365740,0.770899,0.578158,0.334044,1.248301,0.262756,0.254032,0.556946,1.712224
6,2018,10152.9,4313.064,1701.1,3745.5,476.6,1357.1,783.9,1010.6,438.7,1458.9,1895.8,1001.9,313.1,204.3,561.6,4659.2,0.454172,0.351190,0.775678,0.588972,0.362328,1.243946,0.280172,0.269817,0.551315,1.784010
7,2019,10354.7,4459.564,1737.8,3808.3,555.2,1535.2,805.6,1030.6,398.0,1476.4,1873.4,1038.1,316.6,195.3,595.8,4833.1,0.456319,0.361647,0.781681,0.597364,0.403120,1.269096,0.319484,0.270619,0.562271,1.742363
8,2020,9662.4,4057.904,1654.0,3545.7,508.6,1378.7,710.1,904.1,394.6,1350.5,1745.3,964.4,288.5,182.8,509.0,4525.8,0.466481,0.368898,0.785422,0.583071,0.388837,1.276419,0.307497,0.254985,0.569262,1.894695
9,2021,9645.7,4051.844,1615.4,3507.5,506.8,1419.9,708.8,905.1,384.1,1339.6,1725.0,977.5,296.1,182.8,530.2,4444.1,0.460556,0.356927,0.783118,0.605113,0.404818,1.267028,0.313730,0.258047,0.564341,1.843644


In [24]:
change_per48_df = change_df.copy()
for col in change_per48_df.columns[2:18]: # calculate the per 48 minute stats, not including % stats
    change_per48_df[col] = (change_per48_df[col]/change_per48_df['MIN'])*48*5

change_per48_df.drop(columns='MIN', inplace=True)

fig = go.Figure()
for col in change_per48_df.columns[1:]:
    fig.add_trace(go.Scatter(x=change_per48_df['season_start_year'],
                             y=change_per48_df[col], name=col))
fig.show()

In [25]:
change_per100_df = change_df.copy()

for col in change_per100_df.columns[3:18]:
    change_per100_df[col] = (change_per100_df[col]/change_per100_df['POSS_est'])*100

change_per100_df.drop(columns=['MIN','POSS_est'], inplace=True)
change_per100_df

fig = go.Figure()
for col in change_per100_df.columns[1:]:
    fig.add_trace(go.Scatter(x=change_per100_df['season_start_year'],
                             y=change_per100_df[col], name=col))
fig.show()

In [26]:

rs_change_df = rs_df.groupby('season_start_year')[total_cols].sum().reset_index()
playoffs_change_df = playoffs_df.groupby('season_start_year')[total_cols].sum().reset_index()

for i in [rs_change_df,playoffs_change_df]:
    i['POSS_est'] = i['FGA']-i['OREB']+i['TOV']+0.44*i['FTA']
    i['POSS_per_48'] = (i['POSS_est']/i['MIN'])*48*5
    
    i['FG%'] = i['FGM']/i['FGA']
    i['3PT%'] = i['FG3M']/i['FG3A']
    i['FT%'] = i['FTM']/i['FTA']
    i['AST%'] = i['AST']/i['FGM']
    i['FG3A%'] = i['FG3A']/i['FGA']
    i['PTS/FGA'] = i['PTS']/i['FGA']
    i['FG3M/FGM'] = i['FG3M']/i['FGM']
    i['FTA/FGA'] = i['FTA']/i['FGA']
    i['TRU%'] = 0.5*i['PTS']/(i['FGA']+0.475*i['FTA'])
    i['AST_TOV'] = i['AST']/i['TOV']
    for col in total_cols:
        i[col] = 100*i[col]/i['POSS_est']
    i.drop(columns=['MIN','POSS_est'], inplace=True)
    
rs_change_df

,season_start_year,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PTS,POSS_per_48,FG%,3PT%,FT%,AST%,FG3A%,PTS/FGA,FG3M/FGM,FTA/FGA,TRU%,AST_TOV
0,2012,39.736744,86.658749,7.610868,20.739616,18.070805,23.788971,11.591553,32.090826,43.662350,22.992834,8.041483,5.262515,14.465657,105.145147,96.911494,0.458543,0.366972,0.759630,0.578629,0.239325,1.213324,0.191532,0.274513,0.536682,1.589477
1,2013,39.172867,85.660814,8.018098,22.115446,18.929667,24.860914,10.807967,32.431300,43.239267,22.896165,7.895995,4.750917,14.208351,105.230597,97.415691,0.457302,0.362556,0.761423,0.584490,0.258175,1.228457,0.204685,0.290225,0.539812,1.611458
2,2014,39.387593,86.715518,8.025314,22.516697,18.104991,23.994656,11.380280,33.741792,45.037089,22.797509,7.951417,5.017669,14.107114,104.868543,96.552106,0.454216,0.356416,0.754543,0.578799,0.259662,1.209340,0.203752,0.276705,0.534427,1.616029
3,2015,39.242430,86.350104,8.805139,24.706848,18.098440,23.777882,10.816076,34.291589,45.071235,22.506474,8.080181,5.074703,14.003704,105.413940,97.912895,0.454457,0.356385,0.761146,0.573524,0.286124,1.220774,0.224378,0.275366,0.539784,1.607180
4,2016,39.626915,86.209107,9.932548,27.557569,18.394268,23.629422,10.000214,33.419518,43.387679,23.269728,7.720963,4.700955,13.394161,107.573524,99.715254,0.459660,0.360429,0.778448,0.587220,0.319660,1.247821,0.250652,0.274094,0.552038,1.737304
5,2017,40.215301,86.490179,10.566795,28.891539,16.937612,21.971249,9.736821,33.825272,43.539039,23.250792,7.800216,4.856883,13.579292,107.965750,99.243371,0.464970,0.365740,0.770899,0.578158,0.334044,1.248301,0.262756,0.254032,0.556946,1.712224
6,2018,40.487306,86.875025,11.097109,30.898682,17.791462,22.980940,10.122729,33.720776,43.814634,23.655788,7.325896,4.778072,13.136090,109.837921,103.577193,0.466041,0.359145,0.774183,0.584277,0.355668,1.264321,0.274089,0.264529,0.561595,1.800824
7,2019,39.558565,85.858845,12.119820,33.375479,17.629775,22.636618,9.121960,32.702349,41.806960,23.847559,7.248298,4.507199,13.303003,108.828559,104.905352,0.460740,0.363135,0.778817,0.602842,0.388725,1.267529,0.306377,0.263649,0.563229,1.792645
8,2020,40.987014,86.923848,12.468681,33.574052,17.065502,21.776067,9.495652,33.315187,42.826527,24.858918,7.479640,4.573288,12.990334,111.543512,102.019227,0.471528,0.371379,0.783682,0.606507,0.386247,1.283233,0.304211,0.250519,0.573385,1.913647
9,2021,40.459773,87.307931,12.860201,35.716159,16.642379,21.342292,9.483256,33.487375,42.998441,25.048986,7.484899,4.441676,12.784716,110.406235,102.202125,0.463415,0.360067,0.779784,0.619108,0.409083,1.264561,0.317852,0.244448,0.566502,1.959291


In [32]:
comp_change_df = round(100*(playoffs_change_df-rs_change_df)/rs_change_df,3)
comp_change_df['season_start_year'] = list(range(2012,2022))
comp_change_df

,season_start_year,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PTS,POSS_per_48,FG%,3PT%,FT%,AST%,FG3A%,PTS/FGA,FG3M/FGM,FTA/FGA,TRU%,AST_TOV
0,2012,-5.896,-2.288,-1.289,5.640,8.666,10.829,-6.963,1.463,-0.846,-10.572,-7.955,-2.529,0.289,-3.100,-3.264,-3.693,-6.558,-1.951,-4.968,8.113,-0.832,4.896,13.424,-2.344,-10.829
1,2013,-1.115,0.799,4.519,7.908,6.638,7.014,8.186,0.819,2.599,-8.321,-9.111,-0.042,-3.990,0.604,-4.998,-1.898,-3.140,-0.351,-7.288,7.053,-0.193,5.697,6.166,-0.933,-4.511
2,2014,-3.602,0.456,6.962,12.348,2.216,4.145,-1.525,1.860,0.959,-3.358,-2.815,4.672,-7.138,-1.946,-0.278,-4.040,-4.794,-1.853,0.253,11.838,-2.392,10.958,3.672,-2.806,4.071
3,2015,-4.382,0.708,1.590,6.089,1.086,4.233,0.461,-1.419,-0.949,-8.928,-6.954,3.545,-7.172,-3.110,-3.528,-5.054,-4.241,-3.019,-4.755,5.343,-3.791,6.245,3.500,-4.179,-1.891
4,2016,-1.288,-0.416,3.236,5.483,5.065,5.621,1.343,-1.871,-1.333,-6.103,-1.255,0.445,-0.686,0.149,-2.872,-0.876,-2.130,-0.527,-4.878,5.923,0.567,4.583,6.062,-0.131,-5.455
5,2017,-6.603,0.334,3.776,12.428,11.375,10.327,5.362,0.554,1.534,-3.387,-8.467,-4.001,-5.636,-2.962,-0.081,-6.914,-7.696,0.950,3.443,12.054,-3.285,11.113,9.959,-4.311,2.384
6,2018,-6.418,-2.675,17.618,17.617,5.999,4.324,-15.397,0.367,-3.281,-5.995,-6.807,-13.233,2.498,-2.084,-2.817,-3.846,0.001,1.607,0.452,20.850,0.608,25.685,7.191,-0.193,-8.286
7,2019,2.067,2.662,4.319,3.832,3.417,2.187,10.838,1.591,3.619,-8.089,-10.544,-2.622,-11.389,2.463,-5.835,-0.580,0.468,1.204,-9.950,1.140,-0.195,2.206,-0.463,-0.143,3.723
8,2020,-5.096,-1.813,-4.320,1.089,10.684,10.079,-0.231,-2.858,-2.220,-9.050,-6.181,1.154,4.528,-2.736,-3.329,-3.344,-5.351,0.549,-4.166,2.956,-0.940,0.818,12.112,-2.200,-12.990
9,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Interesting: More FTA attempts in playoofs, less asists and blocks, worst ball handeling (AST/TOV)

In [28]:
fig = go.Figure()
for col in comp_change_df.columns[1:]:
    fig.add_trace(go.Scatter(x=comp_change_df['season_start_year'],
                             y=comp_change_df[col], name=col))
fig.show()

In the playoffs: Slower paste of play, way less assists, for the most part less TOV, less blocks and steals (smarter shot decision making), less FGA and FGM